# Modeling process

In [99]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

# Editing display option for jupyter notebook to show all rows and column of dataframes
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Loading scraped data and initial data cleaning

In [101]:
# Loading data
df = pd.read_csv('data_trojmiasto.csv')

In [102]:
# Droping unusable 
df.drop(df.columns[[0, 14, 15, 16]],axis=1,inplace=True)

In [104]:
# Renaming every column to english and standardized names, lower cased and no spaces
df.rename(columns = {'Cena':'price',
                     'Liczba pokoi':'rooms',
                     'Piętro':'floor',
                     'Rok budowy':'year',
                     'Jestem':'landlord_type',
                     'Rodzaj nieruchomości':'local_type',
                     'Umeblowane':'furnished',
                     'Liczba pięter w budynku':'levels',
                     'Powierzchnia':'area_size',
                     'Niezależne pokoje':'indep_rooms',
                     'Dostępne od':'availability_date',
                     'Kaucja':'deposit',
                     'Typ ogrzewania':'heating_type',
                     'Liczba osób w pokoju':'people_limit',
                     'Powierzchnia działki':'land_area'}, inplace = True)

In [105]:
# Converting values of selected columns to desirable types and formats
df['price'] = df['price'].str.replace(' ', '')
df['price'] = df['price'].str.extract('(\d+)')
df['price'] = df['price'].astype('int')
df['area_size'] = df['area_size'].str.replace(',', '.')
df['area_size'] = df['area_size'].str.replace(' ', '')
df['area_size'] = df['area_size'].astype('float')
# df['rooms'] = df['rooms'].astype('int')
# df['year'] = df['year'].astype('int')

In [106]:
df.head()

,price,rooms,floor,year,landlord_type,local_type,furnished,levels,area_size,indep_rooms,availability_date,deposit,heating_type,people_limit,land_area
0,4500,1.0,Parter,1900.0,Osobą prywatną,Lokal handlowy,Nie,3.0,50.0,Tak,Zaraz,NaN,NaN,NaN,NaN
1,2500,1.0,2,2021.0,Osobą prywatną,Mieszkanie,Tak,17.0,31.0,NaN,02. 01. 2023,2500.0,Miejskie,NaN,NaN
2,2400,2.0,4,NaN,Firmą,Mieszkanie,Tak,4.0,44.0,NaN,NaN,NaN,C. o. miejskie,NaN,NaN
3,2300,2.0,2,2022.0,Firmą,Mieszkanie,Tak,4.0,38.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2200,2.0,Parter,2020.0,Firmą,Mieszkanie,Tak,4.0,30.0,NaN,01-01-2023,NaN,NaN,NaN,NaN


In [107]:
# Initial length of data frame
len(df)

2859

In [108]:
# Summary of missing data
print('Braki:')
print('%       ','N     ','VAR')
for i in df.columns:
    print("{:4.2f}".format(round(len(df[df[i].isna()]) / len(df), 2)), "  ", "{:4}".format(len(df[df[i].isna()])), "  ",i)

Braki:

%        N      VAR

0.00       0    price

0.32     906    rooms

0.18     509    floor

0.29     816    year

0.00       0    landlord_type

0.00       2    local_type

0.06     175    furnished

0.28     790    levels

0.00       0    area_size

0.86    2452    indep_rooms

0.73    2077    availability_date

0.76    2175    deposit

0.39    1124    heating_type

0.92    2627    people_limit

0.91    2605    land_area


In [109]:
# Deleting every variable with missing data ratio greater than 0.38
dfm = df[['price','rooms','floor','year','landlord_type','local_type','furnished','levels','area_size']]

In [110]:
# Deleting rows with NaN values in any of the remaining columns
dfm = dfm[~(dfm['rooms'].isna()) & ~(dfm['floor'].isna()) & ~(dfm['furnished'].isna()) & ~(dfm['local_type'].isna()) & ~(dfm['year'].isna()) & ~(dfm['levels'].isna())]

In [111]:
# Deleting rows where value of 'price' variable is greater than 9th decile or lover than 1th decile
dfm = dfm[(dfm.price >= dfm.price.quantile(.10)) & (dfm.price <= dfm.price.quantile(.90))]

In [112]:
# Deleting detected outlier value of variable 'levels'
dfm = dfm[dfm['levels'] < 3000]

In [113]:
# Summary of missing data on changed dataset
print('Braki:')
print('%       ','N     ','VAR')
for i in dfm.columns:
    print("{:4.2f}".format(round(len(dfm[dfm[i].isna()]) / len(dfm), 2)), "  ", "{:4}".format(len(dfm[dfm[i].isna()])), "  ",i)

Braki:

%        N      VAR

0.00       0    price

0.00       0    rooms

0.00       0    floor

0.00       0    year

0.00       0    landlord_type

0.00       0    local_type

0.00       0    furnished

0.00       0    levels

0.00       0    area_size


### Analytical process of defining standardized categorical variables

In [114]:
dfm.head()

,price,rooms,floor,year,landlord_type,local_type,furnished,levels,area_size
0,4500,1.0,Parter,1900.0,Osobą prywatną,Lokal handlowy,Nie,3.0,50.0
1,2500,1.0,2,2021.0,Osobą prywatną,Mieszkanie,Tak,17.0,31.0
3,2300,2.0,2,2022.0,Firmą,Mieszkanie,Tak,4.0,38.0
4,2200,2.0,Parter,2020.0,Firmą,Mieszkanie,Tak,4.0,30.0
6,2950,2.0,6,1972.0,Osobą prywatną,Mieszkanie,Tak,11.0,38.0


In [115]:
# Data frame length after changes
len(dfm)

1143

In [ ]:
# Code cells below from 116 to 130 show a repeated proces of describing variable structure 
# to later define artificial category ranges for all variables except 'price' and 'area_size'

In [116]:
dfm.price.describe()

count    1143.000000
mean     3354.982502
std      1366.709336
min      1950.000000
25%      2500.000000
50%      2900.000000
75%      3700.000000
max      8592.000000
Name: price, dtype: float64

In [117]:
dfm.price.quantile(.90)

5500.0

In [118]:
len(dfm[(dfm.price >= dfm.price.quantile(.10)) & (dfm.price <= dfm.price.quantile(.90))])

950

In [119]:
dfm.groupby(['rooms'])['rooms'].count()

rooms
1.0    201
2.0    609
3.0    251
4.0     60
5.0     18
6.0      2
7.0      1
9.0      1
Name: rooms, dtype: int64

In [120]:
dfm.groupby(['floor'])['floor'].count()

floor
-2          1
1         260
10         17
11          7
12         10
13          2
14          5
15          6
16          7
17          1
2         205
3         149
4         119
5          41
6          21
7          21
8          13
9          14
Parter    244
Name: floor, dtype: int64

In [121]:
dfm.floor.unique()

array(['Parter', '2', '6', '17', '1', '4', '3', '5', '9', '8', '16', '15',
       '11', '7', '12', '10', '14', '-2', '13'], dtype=object)

In [122]:
dfm.year.describe()

count    1143.000000
mean     1998.595801
std        69.750991
min        19.000000
25%      1990.000000
50%      2012.000000
75%      2020.000000
max      2023.000000
Name: year, dtype: float64

In [123]:
dfm.landlord_type.unique()

array(['Osobą prywatną', 'Firmą'], dtype=object)

In [124]:
dfm.groupby(['local_type'])['local_type'].count()

local_type
Biuro                    85
Dom piętro domu           1
Lokal biurowy             2
Lokal gastronomiczny      1
Lokal handlowy            6
Lokal usługowy           83
Magazyn                   4
Mieszkanie              946
Obiekt                    2
Pokój                     2
Sklep                    11
Name: local_type, dtype: int64

In [125]:
dfm.furnished.unique()

array(['Nie', 'Tak'], dtype=object)

In [126]:
dfm.levels.describe()

count    1143.000000
mean        5.186352
std         3.859315
min         1.000000
25%         3.000000
50%         4.000000
75%         5.000000
max        20.000000
Name: levels, dtype: float64

In [127]:
dfm.groupby(['levels'])['levels'].count()

levels
1.0      49
2.0     104
3.0     246
4.0     331
5.0     157
6.0      50
7.0      24
8.0      16
9.0       5
10.0     66
11.0      9
12.0      4
13.0      2
14.0      3
16.0     17
17.0     46
18.0     13
20.0      1
Name: levels, dtype: int64

In [129]:
dfm.area_size.describe()

count    1143.000000
mean       54.830700
std        26.902303
min        15.000000
25%        40.000000
50%        48.000000
75%        62.380000
max       265.000000
Name: area_size, dtype: float64

### Transforming variables according to the previous analysis

In [131]:
# Local type
# After analysis following categories were defined: mieszkanie, dom, biznes, parking, inne
mieszkalne = ['Mieszkanie','Lokal hotelowy','Pokój','Dom piętro domu','Dom bliźniak','Pokój']
biznes = ['Lokal handlowy','Lokal usługowy', 'Biuro', 'Magazyn', 'Sklep','Lokal gastronomiczny','Lokal biurowy']
inne = ['Obiekt'] 

dfm['s_local_type'] = np.nan

for i in range(0, len(dfm)):
    if dfm.iloc[i, 5] in mieszkalne:
        dfm.iloc[i, 9] = 'mieszkalne'
    if dfm.iloc[i, 5] in biznes:
        dfm.iloc[i, 9] = 'komercyjne'
    if dfm.iloc[i, 5] in inne:
        dfm.iloc[i, 9] = 'inne'

In [132]:
# Floor
# After analysis following categories were defined: parter, 1, 2, 3, 4, 5-10, >10
parter = ['Parter','-2']
plus5 = ['6','5','9','8','7','10']
plus10 = ['11','12','14','13','17','16','15']

dfm['s_floor'] = np.nan

for i in range(0, len(dfm)):
    if dfm.iloc[i, 2] in parter:
        dfm.iloc[i, 10] = '<=0'
    if dfm.iloc[i, 2] == '1':
        dfm.iloc[i, 10] = '1'
    if dfm.iloc[i, 2] == '2':
        dfm.iloc[i, 10] = '2'
    if dfm.iloc[i, 2] == '3':
        dfm.iloc[i, 10] = '3'
    if dfm.iloc[i, 2] == '4':
        dfm.iloc[i, 10] = '4'
    if dfm.iloc[i, 2] in plus5:
        dfm.iloc[i, 10] = '5-10'
    if dfm.iloc[i, 2] in plus10:
        dfm.iloc[i, 10] = '>10'

In [133]:
# Rooms
# After analysis following categories were defined: 1, 2, 3, 4+
dfm['s_rooms'] = np.nan

for i in range(0, len(dfm)):
    if dfm.iloc[i, 1] == 1:
        dfm.iloc[i, 11] = '1'
    if dfm.iloc[i, 1] == 2:
        dfm.iloc[i, 11] = '2'
    if dfm.iloc[i, 1] == 3:
        dfm.iloc[i, 11] = '3'
    if dfm.iloc[i, 1] >= 4:
        dfm.iloc[i, 11] = '4+'

In [134]:
# Year
# After analysis following categories were defined: <2005, 2005-2009, 2010-2014, 2015-2019, 2020+
dfm['s_year'] = np.nan

for i in range(0, len(dfm)):
    if dfm.iloc[i, 3] < 1990:
        dfm.iloc[i, 12] = '<1990'
    if (dfm.iloc[i, 3] >= 1990) & (dfm.iloc[i, 3] < 2005):
        dfm.iloc[i, 12] = '1990-2005'
    if (dfm.iloc[i, 3] >= 2005) & (dfm.iloc[i, 3] < 2015):
        dfm.iloc[i, 12] = '2005-2015'
    if (dfm.iloc[i, 3] >= 2015) & (dfm.iloc[i, 3] < 2020):
        dfm.iloc[i, 12] = '2015-2020'
    if dfm.iloc[i, 3] >= 2020:
        dfm.iloc[i, 12] = '>=2020'

In [136]:
# Levels
# After analysis following categories were defined: <=2, 3, 4, 5, >6
dfm['s_levels'] = np.nan

for i in range(0, len(dfm)):
    if dfm.iloc[i, 7] <= 2:
        dfm.iloc[i, 13] = '<=2'
    if dfm.iloc[i, 7] == 3:
        dfm.iloc[i, 13] = '3'
    if dfm.iloc[i, 7] == 4:
        dfm.iloc[i, 13] = '4'
    if dfm.iloc[i, 7] == 5:
        dfm.iloc[i, 13] = '5'
    if dfm.iloc[i, 7] >= 6:
        dfm.iloc[i, 13] = '>6'

In [138]:
# Landlord type
# string conversion to lower case and no spaces
dfm['s_landlord_type'] = np.nan

for i in range(0, len(dfm)):
    if dfm.iloc[i, 4] == 'Osobą prywatną':
        dfm.iloc[i, 14] = 'osoba_prywatna'
    if dfm.iloc[i, 4] == 'Firmą':
        dfm.iloc[i, 14] = 'firma'

In [139]:
# Defining new data frame with only standardized variables for modeling process
dfs = dfm[['price','s_rooms','s_floor','s_year','s_landlord_type','s_local_type','furnished','s_levels','area_size']]

In [140]:
dfs.head(1)

,price,s_rooms,s_floor,s_year,s_landlord_type,s_local_type,furnished,s_levels,area_size
0,4500,1,<=0,<1990,osoba_prywatna,komercyjne,Nie,3,50.0


#### Dummy coding

In [141]:
# Generating dummy variables (binary column for every category of every variable)
dfr = pd.get_dummies(dfs)

In [142]:
dfr.head()

,price,area_size,s_rooms_1,s_rooms_2,s_rooms_3,s_rooms_4+,s_floor_1,s_floor_2,s_floor_3,s_floor_4,s_floor_5-10,s_floor_<=0,s_floor_>10,s_year_1990-2005,s_year_2005-2015,s_year_2015-2020,s_year_<1990,s_year_>=2020,s_landlord_type_firma,s_landlord_type_osoba_prywatna,s_local_type_inne,s_local_type_komercyjne,s_local_type_mieszkalne,furnished_Nie,furnished_Tak,s_levels_3,s_levels_4,s_levels_5,s_levels_<=2,s_levels_>6
0,4500,50.0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,1,0,0,0,0
1,2500,31.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,1
3,2300,38.0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,1,0,0,0
4,2200,30.0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,1,0,1,0,0,0
6,2950,38.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1


## Estimating price regression model

In [146]:
import statsmodels.api as sm
# Defining X and y variables for regression model
X = dfr.drop('price', axis=1)
y = dfr['price']

# Adding constant to the model
X = sm.add_constant(X)

# Estimating regression model
model = sm.OLS(y, X).fit()

In [147]:
# Estimation summary
print(model.summary())

                            OLS Regression Results                            


Dep. Variable:                  price   R-squared:                       0.629

Model:                            OLS   Adj. R-squared:                  0.622

Method:                 Least Squares   F-statistic:                     86.30

Date:                Thu, 12 Jan 2023   Prob (F-statistic):          6.69e-223

Time:                        22:13:38   Log-Likelihood:                -9307.4

No. Observations:                1143   AIC:                         1.866e+04

Df Residuals:                    1120   BIC:                         1.878e+04

Df Model:                          22                                         

Covariance Type:            nonrobust                                         


                                     coef    std err          t      P>|t|      [0.025      0.975]

--------------------------------------------------------------------------------------------------

In [148]:
# Assigning model parameters to 'params' object
params = model.params

In [158]:
# Converting 'params' object to list
lparams = params.values.tolist()

# Saving params list to text file params.txt
with open('params.txt', 'w') as fp:
    for item in lparams:
        fp.write("%s\n" % item)
    print('Done')

Done
